In [1]:
# the TestEnv environment is used to simply simulate the network
from flow.envs import TestEnv

import numpy as np

# the Experiment class is used for running simulations
from flow.core.experiment import Experiment

# all other imports are standard
from flow.core.params import VehicleParams
from flow.core.params import NetParams
from flow.core.params import InitialConfig
from flow.core.params import EnvParams
from flow.core.params import SumoParams

In [2]:
from flow.networks import Network

We are finally ready to test our network in simulation. In order to do so, we create an `Experiment` object and run the simulation for a number of steps. This is done in the cell below.

## 2. Customizing the Network

While the above example does allow you to view the network within Flow, the simulation is limited for two reasons. For one, vehicles are placed on all edges within the network; if we wished to simulate traffic solely on the on the bridge and do not care about the artireols, for instance, this would result in unnecessary computational burdens. Next, as you may have noticed if you ran the above example to completion, routes in the base network class are defaulted to consist of the vehicles' current edges only, meaning that vehicles exit the network as soon as they reach the end of the edge they are originated on. In the next subsections, we discuss how the network can be modified to resolve these issues.

### 2.1 Specifying Traversable Edges

In order to limit the edges vehicles are placed on to the road sections edges corresponding to the westbound Bay Bridge, we define an `EDGES_DISTRIBUTION` variable. This variable specifies the names of the edges within the network that vehicles are permitted to originated in, and is assigned to the network via the `edges_distribution` component of the `InitialConfig` input parameter, as seen in the code snippet below. Note that the names of the edges can be identified from the .osm file or by right clicking on specific edges from the SUMO gui (see the figure below).

<!-- <img src="img/osm_edge_name.png" width=600>
<center> **Figure 2**: Name of an edge from SUMO </center> -->

In [3]:
# we define an EDGES_DISTRIBUTION variable with the edges within the music row corridor:
EDGES_DISTRIBUTION = list(np.loadtxt('music_row_16th_ave_edges.txt',dtype=str,comments=None))

# the above variable is added to initial_config
new_initial_config = InitialConfig(
    edges_distribution=EDGES_DISTRIBUTION
)

### 2.2 Creating Custom Routes

Next, we choose to specify the routes of vehicles so that they can traverse the entire Bay Bridge, instead of the only the edge they are currently on. In order to this, we create a new network class that inherits all its properties from `Network` and simply redefine the routes by modifying the `specify_routes` variable. This method was originally introduced in Tutorial 07: Creating Custom Network. The new network class looks as follows:

In [4]:
from flow.networks.base import Network
from flow.core.params import InitialConfig
from flow.core.params import TrafficLightParams

ADDITIONAL_NET_PARAMS = {}

# we create a new network class to specify the expected routes
class MusicRowCorridor(Network):
    
    def __init__(self,
                 name,
                 vehicles,
                 net_params,
                 initial_config=InitialConfig(),
                 traffic_lights=TrafficLightParams()):
        
        for p in ADDITIONAL_NET_PARAMS.keys():
            if p not in net_params.additional_params:
                raise KeyError('Network parameter "{}" not supplied'.format(p))
                
        self.NET_EDGES = list(np.loadtxt('music_row_16th_ave_edges.txt',dtype=str,comments=None))
    
        super().__init__(name, vehicles, net_params, initial_config,
                         traffic_lights)
    
    
    def specify_routes(self, net_params):
        
        route_dict = dict.fromkeys(self.NET_EDGES)
        
        
        for i in range(len(self.NET_EDGES)):
            edge = self.NET_EDGES[i]
            route_dict[edge] = self.NET_EDGES[i:]      
        
    
        return route_dict

### 2.3 Rerunning the Simulation

We are now ready to rerun the simulation with fully defined vehicle routes and a limited number of traversable edges. If we run the cell below, we can see the new simulation in action.

In [5]:
import os
import numpy as np

from flow.controllers import IDMController

from flow.controllers import SimLaneChangeController, ContinuousRouter

from flow.core.params import SumoParams
from flow.core.params import EnvParams
from flow.core.params import NetParams
from flow.core.params import SumoLaneChangeParams
from flow.core.params import VehicleParams
from flow.core.params import InitialConfig
from flow.core.params import InFlows
import flow.config as config
from flow.envs import TestEnv

HORIZON = 2000

# NET_TEMPLATE = os.path.join(config.PROJECT_PATH,
#         "exp_configs/non_rl/templates/sumo/music_row.net.xml")

# NET_TEMPLATE = os.path.join(config.PROJECT_PATH,'tutorials/networks/music_row.net.xml')
NET_TEMPLATE = os.path.join(config.PROJECT_PATH,'examples/exp_configs/templates/sumo/music_row.net.xml')

edges_distribution = EDGES_DISTRIBUTION

# the inflow rate of vehicles (in veh/hr)
INFLOW_RATE = 1000
# the speed of inflowing vehicles from the main edge (in m/s)
INFLOW_SPEED = 10.0

vehicles = VehicleParams()
vehicles.add(
    "human",
    num_vehicles=0,
    lane_change_controller=(SimLaneChangeController, {}),
    
    lane_change_params=SumoLaneChangeParams(
        lane_change_mode=1621,
    ),
    
    routing_controller=(ContinuousRouter, {}),
    
    acceleration_controller=(IDMController, {
        "a": 1.3,
        "b": 2.0,
        "noise": 0.3,
    })
)

inflow = InFlows()
# main highway
for lane in [0, 1]:
    inflow.add(
        veh_type="human",
        edge='19449685#0',
        vehs_per_hour=INFLOW_RATE,
        depart_lane=lane,
        depart_speed=INFLOW_SPEED)
    
    


In [6]:
flow_params = dict(
    # name of the experiment
    exp_tag='music_row_corridor',

    # name of the flow environment the experiment is running on
    env_name=TestEnv,

    # name of the network class the experiment is running on
    network=MusicRowCorridor,

    # simulator that is used by the experiment
    simulator='traci',

    # simulation-related parameters
    sim=SumoParams(
        sim_step=0.2,
        render=True,
        color_by_speed=True,
        use_ballistic=True
    ),

    # environment related parameters (see flow.core.params.EnvParams)
    env=EnvParams(
        horizon=HORIZON,
        warmup_steps=0,
        sims_per_step=1
    ),

    # network-related parameters (see flow.core.params.NetParams and the
    # network's documentation or ADDITIONAL_NET_PARAMS component)
    net=NetParams(
        inflows=inflow,
        template=NET_TEMPLATE,
        additional_params={}
    ),

    # vehicles to be placed in the network at the start of a rollout (see
    # flow.core.params.VehicleParams)
    veh=vehicles,

    # parameters specifying the positioning of vehicles upon initialization/
    # reset (see flow.core.params.InitialConfig)
    initial=InitialConfig(
        edges_distribution=edges_distribution,
    ),
)

# number of time steps
# exp = Experiment(flow_params)

# run the sumo simulation
# _ = exp.run(1)

## 3. Other Tips

This tutorial introduces how to incorporate OpenStreetMap files in Flow. This feature, however, does not negate other features that are introduced in other tutorials and documentation. For example, if you would like to not have vehicles be originated side-by-side within a network, this can still be done by specifying a "random" spacing for vehicles as follows:

    initial_config = InitialConfig(
        spacing="random",
        edges_distribution=EDGES_DISTRIBUTION
    )

In addition, inflows of vehicles can be added to networks imported from OpenStreetMap as they are for any other network (see the tutorial on adding inflows for more on this).

# 4. Start layering in RL:

In [7]:
import flow.envs as flowenvs

print(flowenvs.__all__)

from flow.envs import MusicRow_POEnv

env_name = MusicRow_POEnv

['Env', 'AccelEnv', 'LaneChangeAccelEnv', 'LaneChangeAccelPOEnv', 'TrafficLightGridTestEnv', 'MergePOEnv', 'MusicRow_POEnv', 'BottleneckEnv', 'BottleneckAccelEnv', 'WaveAttenuationEnv', 'WaveAttenuationPOEnv', 'TrafficLightGridEnv', 'TrafficLightGridPOEnv', 'TrafficLightGridBenchmarkEnv', 'TrafficLightGridGenericObserver', 'BottleneckDesiredVelocityEnv', 'TestEnv', 'BayBridgeEnv', 'BottleNeckAccelEnv', 'DesiredVelocityEnv', 'PO_TrafficLightGridEnv', 'GreenWaveTestEnv']


In [8]:
from flow.controllers import RLController

import json

import ray
try:
    from ray.rllib.agents.agent import get_agent_class
except ImportError:
    from ray.rllib.agents.registry import get_agent_class
from ray.tune import run_experiments
from ray.tune.registry import register_env

from flow.utils.registry import make_create_env
from flow.utils.rllib import FlowParamsEncoder

/opt/anaconda3/envs/flow/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/anaconda3/envs/flow/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/anaconda3/envs/flow/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/anaconda3/envs/flow/lib/python3.6/site-packages/tensorflow/python/frame

In [9]:
# number of parallel workers
N_CPUS = 2
# number of rollouts per training iteration
N_ROLLOUTS = 1

ray.init(num_cpus=N_CPUS)

2025-01-31 15:44:54,466	INFO node.py:498 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2025-01-31_15-44-54_464873_96330/logs.
2025-01-31 15:44:54,585	INFO services.py:409 -- Waiting for redis server at 127.0.0.1:52814 to respond...
2025-01-31 15:44:54,712	INFO services.py:409 -- Waiting for redis server at 127.0.0.1:58062 to respond...
2025-01-31 15:44:54,718	INFO services.py:809 -- Starting Redis shard with 3.44 GB max memory.
2025-01-31 15:44:54,749	INFO node.py:512 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2025-01-31_15-44-54_464873_96330/logs.
2025-01-31 15:44:54,752	WARNING services.py:1330 -- WARNING: The default object store size of 5.15 GB will use more than 50% of the available memory on this node (5.9 GB). Consider setting the object store memory manually to a smaller size to avoid memory contention with other applications.
2025-01-31 15:44:54,753	INFO services.py:1475 -- Starting the Plasma object store with 5.15 GB memory us

{'node_ip_address': '10.71.76.7',
 'redis_address': '10.71.76.7:52814',
 'object_store_address': '/tmp/ray/session_2025-01-31_15-44-54_464873_96330/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2025-01-31_15-44-54_464873_96330/sockets/raylet',
 'webui_url': None,
 'session_dir': '/tmp/ray/session_2025-01-31_15-44-54_464873_96330'}

In [10]:
# The algorithm or model to train. This may refer to "
#      "the name of a built-on algorithm (e.g. RLLib's DQN "
#      "or PPO), or a user-defined trainable function or "
#      "class registered in the tune registry.")
alg_run = "PPO"

agent_cls = get_agent_class(alg_run)
config = agent_cls._default_config.copy()
config["num_workers"] = N_CPUS - 1  # number of parallel workers
config["train_batch_size"] = HORIZON * N_ROLLOUTS  # batch size
config["gamma"] = 0.999  # discount rate
config["model"].update({"fcnet_hiddens": [16, 16]})  # size of hidden layers in network
config["use_gae"] = True  # using generalized advantage estimation
config["lambda"] = 0.97  
config["sgd_minibatch_size"] = min(16 * 1024, config["train_batch_size"])  # stochastic gradient descent
config["kl_target"] = 0.02  # target KL divergence
config["num_sgd_iter"] = 10  # number of SGD iterations
config["horizon"] = HORIZON  # rollout horizon

# save the flow params for replay
flow_json = json.dumps(flow_params, cls=FlowParamsEncoder, sort_keys=True,
                       indent=4)  # generating a string version of flow_params
config['env_config']['flow_params'] = flow_json  # adding the flow_params to config dict
config['env_config']['run'] = alg_run

# Call the utility function make_create_env to be able to 
# register the Flow env for this experiment
create_env, gym_name = make_create_env(params=flow_params, version=0)

# Register as rllib env with Gym

In [11]:
trials = run_experiments({
    flow_params["exp_tag"]: {
        "run": alg_run,
        "env": gym_name,
        "config": {
            **config
        },
        "checkpoint_freq": 1,  # number of iterations between checkpoints
        "checkpoint_at_end": True,  # generate a checkpoint at the end
        "max_failures": 999,
        "stop": {  # stopping conditions
            "training_iteration": 1,  # number of iterations to stop after
        },
    },
})

2025-01-31 15:44:55,386	INFO trial_runner.py:176 -- Starting a new experiment.
2025-01-31 15:44:55,429	WARNING signature.py:108 -- The function with_updates has a **kwargs argument, which is currently not supported.
2025-01-31 15:44:55,441	ERROR log_sync.py:34 -- Log sync requires cluster to be setup with `ray up`.


== Status ==
Using FIFO scheduling algorithm.
Resources requested: 0/2 CPUs, 0/0 GPUs
Memory usage on this node: 11.4/17.2 GB

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 2/2 CPUs, 0/0 GPUs
Memory usage on this node: 11.4/17.2 GB
Result logdir: /Users/vanderbilt/ray_results/music_row_corridor
Number of trials: 1 ({'RUNNING': 1})
RUNNING trials:
 - PPO_TestEnv-v0_0:	RUNNING

(pid=96348) /opt/anaconda3/envs/flow/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=96348)   _np_qint8 = np.dtype([("qint8", np.int8, 1)])
(pid=96348) /opt/anaconda3/envs/flow/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.


2025-01-31 15:44:59,014	ERROR trial_runner.py:550 -- Error processing event.
Traceback (most recent call last):
  File "/opt/anaconda3/envs/flow/lib/python3.6/site-packages/ray/tune/trial_runner.py", line 498, in _process_trial
    result = self.trial_executor.fetch_result(trial)
  File "/opt/anaconda3/envs/flow/lib/python3.6/site-packages/ray/tune/ray_trial_executor.py", line 342, in fetch_result
    result = ray.get(trial_future[0])
  File "/opt/anaconda3/envs/flow/lib/python3.6/site-packages/ray/worker.py", line 2247, in get
    raise value
ray.exceptions.RayTaskError: ray_PPO:train() (pid=96348, host=Vanderbilts-MacBook-Pro.local)
  File "/opt/anaconda3/envs/flow/lib/python3.6/site-packages/gym/envs/registration.py", line 132, in spec
    return self.env_specs[id]
KeyError: 'TestEnv-v0'

During handling of the above exception, another exception occurred:

ray_PPO:train() (pid=96348, host=Vanderbilts-MacBook-Pro.local)
  File "/opt/anaconda3/envs/flow/lib/python3.6/site-packages/ray

(pid=96348) 2025-01-31 15:44:59,003	WARNING ppo.py:143 -- FYI: By default, the value function will not share layers with the policy model ('vf_share_layers': False).
(pid=96347) /opt/anaconda3/envs/flow/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=96347)   _np_qint8 = np.dtype([("qint8", np.int8, 1)])
(pid=96347) /opt/anaconda3/envs/flow/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=96347)   _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
(pid=96347) /opt/anaconda3/envs/flow/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of typ

2025-01-31 15:45:01,673	ERROR trial_runner.py:550 -- Error processing event.
Traceback (most recent call last):
  File "/opt/anaconda3/envs/flow/lib/python3.6/site-packages/ray/tune/trial_runner.py", line 498, in _process_trial
    result = self.trial_executor.fetch_result(trial)
  File "/opt/anaconda3/envs/flow/lib/python3.6/site-packages/ray/tune/ray_trial_executor.py", line 342, in fetch_result
    result = ray.get(trial_future[0])
  File "/opt/anaconda3/envs/flow/lib/python3.6/site-packages/ray/worker.py", line 2247, in get
    raise value
ray.exceptions.RayTaskError: ray_PPO:train() (pid=96347, host=Vanderbilts-MacBook-Pro.local)
  File "/opt/anaconda3/envs/flow/lib/python3.6/site-packages/gym/envs/registration.py", line 132, in spec
    return self.env_specs[id]
KeyError: 'TestEnv-v0'

During handling of the above exception, another exception occurred:

ray_PPO:train() (pid=96347, host=Vanderbilts-MacBook-Pro.local)
  File "/opt/anaconda3/envs/flow/lib/python3.6/site-packages/ray

...

(pid=96739) 2025-01-31 15:51:34,231	WARNING ppo.py:143 -- FYI: By default, the value function will not share layers with the policy model ('vf_share_layers': False).
== Status ==
Using FIFO scheduling algorithm.
Resources requested: 2/2 CPUs, 0/0 GPUs
Memory usage on this node: 10.9/17.2 GB
Result logdir: /Users/vanderbilt/ray_results/music_row_corridor
Number of trials: 1 ({'RUNNING': 1})
RUNNING trials:
 - PPO_TestEnv-v0_0:	RUNNING, 107 failures: /Users/vanderbilt/ray_results/music_row_corridor/PPO_TestEnv-v0_0_2025-01-31_15-44-55j4kj3xrs/error_2025-01-31_15-51-34.txt

(pid=96742) /opt/anaconda3/envs/flow/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=96742)   _np_qint8 = np.dtype([("qint8", np.int8, 1)])
(pid=96742) /opt/anaconda3/envs/flow/lib/python3.6/site-packages/tensorflow/python/framework

2025-01-31 15:51:37,662	ERROR trial_runner.py:550 -- Error processing event.
Traceback (most recent call last):
  File "/opt/anaconda3/envs/flow/lib/python3.6/site-packages/ray/tune/trial_runner.py", line 498, in _process_trial
    result = self.trial_executor.fetch_result(trial)
  File "/opt/anaconda3/envs/flow/lib/python3.6/site-packages/ray/tune/ray_trial_executor.py", line 342, in fetch_result
    result = ray.get(trial_future[0])
  File "/opt/anaconda3/envs/flow/lib/python3.6/site-packages/ray/worker.py", line 2247, in get
    raise value
ray.exceptions.RayTaskError: ray_PPO:train() (pid=96742, host=Vanderbilts-MacBook-Pro.local)
  File "/opt/anaconda3/envs/flow/lib/python3.6/site-packages/gym/envs/registration.py", line 132, in spec
    return self.env_specs[id]
KeyError: 'TestEnv-v0'

During handling of the above exception, another exception occurred:

ray_PPO:train() (pid=96742, host=Vanderbilts-MacBook-Pro.local)
  File "/opt/anaconda3/envs/flow/lib/python3.6/site-packages/ray

(pid=96742) 2025-01-31 15:51:37,656	WARNING ppo.py:143 -- FYI: By default, the value function will not share layers with the policy model ('vf_share_layers': False).


2025-01-31 15:51:42,733	ERROR worker.py:1616 -- print_logs: Error 61 connecting to 10.71.76.7:52814. Connection refused.
2025-01-31 15:51:42,733	ERROR import_thread.py:89 -- ImportThread: Error 61 connecting to 10.71.76.7:52814. Connection refused.
2025-01-31 15:51:42,734	ERROR worker.py:1716 -- listen_error_messages_raylet: Error 61 connecting to 10.71.76.7:52814. Connection refused.


KeyboardInterrupt: 